In [1]:
#required imports
import pandas as pd
from sklearn.model_selection import train_test_split
import sklearn
import sklearn.compose
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer
from tensorflow.keras.optimizers import Adam
import numpy as np


In [2]:
#1. data loading
data_path = 'life_expectency.csv'

df = pd.read_csv(data_path)

In [3]:
#2 data cleaning

#2.1. remove unnecessary data
df = df.drop(['Country'],axis=1)

#2.2. differentiate features(x) and outputs(y)
x = df.iloc[:,0:-1] #all columns expect last one
y = df.iloc[:,-1] #just last column

#2.3. one-hot encoding
x = pd.get_dummies(x)
x.sample()

#2.4. train,validation,test
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.22,random_state=25)

#2.5. transform column (number standartization of number values of x)
numerical_features = x.select_dtypes(include=['float64', 'int64'])
numerical_columns = numerical_features.columns
ct = ColumnTransformer([("only numeric",StandardScaler(),numerical_columns)],remainder='passthrough')
x_train = ct.fit_transform(x_train)
x_test = ct.transform(x_test)

#2.6 type cast
x_train = np.asarray(x_train).astype(np.float32)
y_train = np.asarray(y_train).astype(np.float32)
x_test = np.asarray(x_test).astype(np.float32)
y_test = np.asarray(y_test).astype(np.float32)


In [4]:
#3. build model

#3.1. Sequential Instance from sklearn
my_model = Sequential()

#3.2. Add input layer
my_model.add(InputLayer(shape=(x_train.shape[1],)))

#3.3. Add hidden layers
my_model.add(Dense(3,activation="relu"))

#3.4 Add output layers
my_model.add(Dense(1))

#3.5 Add optimizer
opt = Adam(learning_rate=0.01)
my_model.compile(loss='mse',metrics=['mae'],optimizer=opt)

/opt/anaconda3/envs/py310/lib/python3.10/site-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


In [5]:
#4. Train model
my_model.fit(x_train,y_train,epochs=100,batch_size=2,verbose=1,validation_split=0.25)

Epoch 1/100
859/859 ━━━━━━━━━━━━━━━━━━━━ 1s 438us/step - loss: 3044.3018 - mae: 48.2139 - val_loss: 62.9803 - val_mae: 6.2117
Epoch 2/100
859/859 ━━━━━━━━━━━━━━━━━━━━ 0s 365us/step - loss: 52.1111 - mae: 5.4429 - val_loss: 23.1399 - val_mae: 3.6262
Epoch 3/100
859/859 ━━━━━━━━━━━━━━━━━━━━ 0s 360us/step - loss: 20.5089 - mae: 3.3297 - val_loss: 17.0581 - val_mae: 3.1152
Epoch 4/100
859/859 ━━━━━━━━━━━━━━━━━━━━ 0s 362us/step - loss: 16.4765 - mae: 2.9982 - val_loss: 17.6988 - val_mae: 3.1673
Epoch 5/100
859/859 ━━━━━━━━━━━━━━━━━━━━ 0s 370us/step - loss: 15.0399 - mae: 2.9639 - val_loss: 17.7821 - val_mae: 3.1428
Epoch 6/100
859/859 ━━━━━━━━━━━━━━━━━━━━ 0s 364us/step - loss: 15.7202 - mae: 2.9630 - val_loss: 17.2340 - val_mae: 3.0639
Epoch 7/100
859/859 ━━━━━━━━━━━━━━━━━━━━ 0s 364us/step - loss: 17.5280 - mae: 3.1254 - val_loss: 18.1701 - val_mae: 3.2164
Epoch 8/100
859/859 ━━━━━━━━━━━━━━━━━━━━ 0s 368us/step - loss: 13.5690 - mae: 2.8049 - val_loss: 16.5648 - val_mae: 3.0349
Epoch 9/100
8

In [6]:
#5. Evaluate
res_mse,res_mae = my_model.evaluate(x_test,y_test,verbose=0)
print("++++++++++++++")
print("MSE result : "+str(res_mse))
print("MAE result : "+str(res_mae))
print("++++++++++++++")


++++++++++++++
MSE result : 8.706700325012207
MAE result : 2.2845051288604736
++++++++++++++


In [7]:
y.describe()

count    2938.000000
mean       69.234717
std         9.509115
min        36.300000
25%        63.200000
50%        72.100000
75%        75.600000
max        89.000000
Name: Life expectancy, dtype: float64